In [8]:
from pyspark.sql import*
from pyspark import SparkConf,SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, monotonically_increasing_id
from pyspark.sql.functions import col, regexp_replace
from pyspark.sql.types import FloatType
import os
from pyspark.sql.functions import dayofmonth, hour, dayofyear
from pyspark.sql.functions import col, to_date, when
from pyspark.sql.types import IntegerType,BooleanType,DateType
import findspark
from pyspark.sql.functions import udf
findspark.init("C:\spark")

In [9]:
os.environ["SPARK_HOME"]=r"C:\spark"
os.environ["JAVA_HOME"]=r"C:\Program Files\Java\jdk-20"
os.environ["HADOOP_HOME"]=r"C:\spark\hadoop\bin"

In [15]:

conf = SparkConf()
conf.set("spark.app.name","teste")
conf.set("spark.master","local[3]")
spark = SparkSession.builder\
    .config(conf=conf)\
    .appName("teste")\
    .master("local[3]")\
    .getOrCreate()


sc = spark.sparkContext

In [11]:
file_path = r"C:\spark_treinamento\data_superstore.csv"


df = spark.read.csv(file_path, header=True, inferSchema=True,sep=";")

In [16]:

columns_to_process = ["Sales", "Profit"]

df = df.select(*[
    udf(
        lambda value, col_name=col_name: float(value.replace(",", ".")) if value else None,
        FloatType()
    )(col(col_name)).alias(col_name)
    for col_name in columns_to_process
])

df

DataFrame[Sales: float, Profit: float]

In [14]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import FloatType
import os
import findspark

findspark.init(r"C:\spark")
os.environ["SPARK_HOME"] = r"C:\spark"
os.environ["HADOOP_HOME"] = r"C:\spark\hadoop\bin"
os.environ["JAVA_HOME"] = r"C:\Program Files\Java\jdk-20"

spark = SparkSession.builder.appName("read_csv").getOrCreate()

file_path = r"C:\spark_treinamento\data_superstore.csv"

df = spark.read.csv(file_path, header=True, inferSchema=True, sep=";")


columns_to_process = ["Sales", "Profit"]


df_corrected = df.select(
    *[col(col_name).cast(FloatType()).alias(col_name) if col_name in columns_to_process else col(col_name)
      for col_name in df.columns]
)


df_corrected = df.withColumn("Sales", regexp_replace(col("Sales"), ",", ".").cast(FloatType()))
df_corrected = df_corrected.withColumn("Profit", regexp_replace(col("Profit"), ",", ".").cast(FloatType()))


df_corrected.show()

+-----+--------------+----------+----------+--------------+----------+------------------+-----------+-------------+---------------+--------------+----------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+
|RowID|       OrderID| OrderDate|  ShipDate|      ShipMode|CustomerID|      CustomerName|    Segment|      Country|           City|         State|PostalCode| Region|      ProductID|       Category|Sub-Category|         ProductName|   Sales|Quantity|Discount|  Profit|
+-----+--------------+----------+----------+--------------+----------+------------------+-----------+-------------+---------------+--------------+----------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+
|    1|CA-2016-152156|08/11/2016|11/11/2016|  Second Class|  CG-12520|       Claire Gute|   Consumer|United States|      Henderson|      Kentucky|     42420|  South|FUR-BO-10001798|      Furniture

In [18]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace
from pyspark.sql.types import FloatType
import os
import findspark

findspark.init(r"C:\spark")
os.environ["SPARK_HOME"] = r"C:\spark"
os.environ["HADOOP_HOME"] = r"C:\spark\hadoop\bin"
os.environ["JAVA_HOME"] = r"C:\Program Files\Java\jdk-20"

spark = SparkSession.builder.appName("read_csv").getOrCreate()

file_path = r"C:\spark_treinamento\data_superstore.csv"

df = spark.read.csv(file_path, header=True, inferSchema=True, sep=";")


columns_to_process = ["Sales", "Profit"]


df_corrected = df.select(
    *[regexp_replace(col(col_name), ",", ".").cast(FloatType()).alias(col_name)
      if col_name in columns_to_process else col(col_name)
      for col_name in df.columns]
)


df_corrected.show(3)


+-----+--------------+----------+----------+------------+----------+---------------+---------+-------------+-----------+----------+----------+------+---------------+---------------+------------+--------------------+------+--------+--------+-------+
|RowID|       OrderID| OrderDate|  ShipDate|    ShipMode|CustomerID|   CustomerName|  Segment|      Country|       City|     State|PostalCode|Region|      ProductID|       Category|Sub-Category|         ProductName| Sales|Quantity|Discount| Profit|
+-----+--------------+----------+----------+------------+----------+---------------+---------+-------------+-----------+----------+----------+------+---------------+---------------+------------+--------------------+------+--------+--------+-------+
|    1|CA-2016-152156|08/11/2016|11/11/2016|Second Class|  CG-12520|    Claire Gute| Consumer|United States|  Henderson|  Kentucky|     42420| South|FUR-BO-10001798|      Furniture|   Bookcases|Bush Somerset Col...|261.96|       2|       0|41.9136|
|   

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace
from pyspark.sql.types import FloatType
import os
import findspark

findspark.init(r"C:\spark")
os.environ["SPARK_HOME"] = r"C:\spark"
os.environ["HADOOP_HOME"] = r"C:\spark\hadoop\bin"
os.environ["JAVA_HOME"] = r"C:\Program Files\Java\jdk-20"

spark = SparkSession.builder.appName("read_csv").getOrCreate()

file_path = r"C:\spark_teste\data_superstore.csv"

df = spark.read.csv(file_path, header=True, inferSchema=True, sep=";")


df_corrected = df.withColumn("Sales", regexp_replace(col("Sales"), ",", ".").cast(FloatType()))
df_corrected = df_corrected.withColumn("Profit", regexp_replace(col("Profit"), ",", ".").cast(FloatType()))


df_corrected.select("Sales", "Profit").show(3)


In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType
from pyspark.sql import SparkSession


spark = SparkSession.builder.appName("Exemplo").getOrCreate()


def process_column(column):
    return udf(
        lambda value: float(value.replace(",", ".")) if value else None,
        FloatType()
    )(column)


columns_to_process = ["Sales", "Values", "Quantity"]


for col_name in columns_to_process:
    df = df.withColumn(col_name, process_column(df[col_name]))


df.show()


In [ ]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import FloatType
from pyspark.sql import SparkSession


spark = SparkSession.builder.appName("Exemplo").getOrCreate()

columns_to_process = ["Sales", "Values", "Quantity"]

df = df.select(*[
    udf(
        lambda value, col_name=col_name: float(value.replace(",", ".")) if value else None,
        FloatType()
    )(col(col_name)).alias(col_name)
    for col_name in columns_to_process
])


df


In [35]:
from random import randint


registros_vendas = {
    "2023": {
        "Janeiro": {"produto1": randint(1, 1000), "produto2": randint(1, 1000)},
        "Fevereiro": {"produto1": randint(1, 1000), "produto2": randint(1, 1000)},
        "Março": {"produto1": randint(1, 1000), "produto2": randint(1, 1000)},
    },
    "2024": {
        "Janeiro": {"produto1": randint(1, 1000), "produto2": randint(1, 1000)},
        "Fevereiro": {"produto1": randint(1, 1000), "produto2": randint(1, 1000)},
        "Março": {"produto1": randint(1, 1000), "produto2": randint(1, 1000)},
    },
}


vendas_por_produto = {
    ano: {
        mes: {
            produto: sum(registros_vendas[ano][mes][produto] for mes in registros_vendas[ano])
            for produto in registros_vendas[ano][mes]
        }
        for mes in registros_vendas[ano]
    }
    for ano in registros_vendas
}


produtos_ordenados = sorted(
    ((ano, mes, produto, vendas)
    for ano, meses in vendas_por_produto.items()
    for mes, produtos in meses.items()
    for produto, vendas in produtos.items()),
    key=lambda x: x[3],
    reverse=True,
)


indice = 0
while indice < len(produtos_ordenados):
    ano, mes, produto, vendas = produtos_ordenados[indice]
    print(f"Ano: {ano}, Mês: {mes}, Produto: {produto}, Vendas: {vendas}")
    indice += 1

Ano: 2024, Mês: Janeiro, Produto: produto1, Vendas: 2212
Ano: 2024, Mês: Fevereiro, Produto: produto1, Vendas: 2212
Ano: 2024, Mês: Março, Produto: produto1, Vendas: 2212
Ano: 2023, Mês: Janeiro, Produto: produto2, Vendas: 1601
Ano: 2023, Mês: Fevereiro, Produto: produto2, Vendas: 1601
Ano: 2023, Mês: Março, Produto: produto2, Vendas: 1601
Ano: 2023, Mês: Janeiro, Produto: produto1, Vendas: 1530
Ano: 2023, Mês: Fevereiro, Produto: produto1, Vendas: 1530
Ano: 2023, Mês: Março, Produto: produto1, Vendas: 1530
Ano: 2024, Mês: Janeiro, Produto: produto2, Vendas: 1108
Ano: 2024, Mês: Fevereiro, Produto: produto2, Vendas: 1108
Ano: 2024, Mês: Março, Produto: produto2, Vendas: 1108


In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType
from pyspark.sql import SparkSession


spark = SparkSession.builder.appName("Exemplo").getOrCreate()


process_sales = udf(
    lambda sales: float(sales.replace(",", ".")) if sales else None,
    FloatType()
)


df = df.withColumn("Sales", process_sales(df["Sales"]))


df.show()


In [ ]:
rdd = sc.parallelize([(1, 'a'), (2, 'b'), (1, 'c')])
grouped = rdd.groupByKey()
for key, values in grouped.collect():
    print(key, list(values))



In [ ]:

lista = [1, 2, 3, 4, 5]
nova_lista = [x * 2 for x in lista]


In [ ]:

rdd = sc.parallelize([1, 2, 3, 4, 5])
novo_rdd = rdd.map(lambda x: x * 2)


In [ ]:

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Exemplo").getOrCreate()
df = spark.createDataFrame([(1,), (2,), (3,), (4,), (5,)], ["valor"])
novo_df = df.select((df.valor * 2).alias("novo_valor"))


In [ ]:

lista = [1, 2, 3, 4, 5]
nova_lista = [x for x in lista if x % 2 == 0]


In [ ]:

rdd = sc.parallelize([1, 2, 3, 4, 5])
novo_rdd = rdd.filter(lambda x: x % 2 == 0)


In [ ]:

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Exemplo").getOrCreate()
df = spark.createDataFrame([(1,), (2,), (3,), (4,), (5,)], ["valor"])
novo_df = df.filter(df.valor % 2 == 0)


In [ ]:

lista = [1, 2, 3, 4, 5]
media = sum(lista) / len(lista)


In [ ]:

rdd = sc.parallelize([1, 2, 3, 4, 5])
media = rdd.aggregate((0, 0), lambda acc, x: (acc[0] + x, acc[1] + 1), lambda acc1, acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1]))
resultado = media[0] / media[1]


In [ ]:

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Exemplo").getOrCreate()
df = spark.createDataFrame([(1,), (2,), (3,), (4,), (5,)], ["valor"])
resultado = df.agg(avg("valor")).collect()[0][0]


In [ ]:

lista = [1, 2, 3, 4, 5]
resultado = sum([x * 2 for x in lista if x % 2 == 0])


In [ ]:

rdd = sc.parallelize([1, 2, 3, 4, 5])
resultado = rdd.filter(lambda x: x % 2 == 0).map(lambda x: x * 2).sum()


In [ ]:
n = 10
fibonacci = [0, 1] + [fibonacci[i - 1] + fibonacci[i - 2] for i in range(2, n)]


In [ ]:
nomes = ["Alice", "Bob", "Charlie", "David"]
iniciais = [nome[0] for nome in nomes if len(nome) > 4]


In [ ]:
matriz = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
diagonal = [linha[i] for i, linha in enumerate(matriz)]


In [ ]:
dados = [("Alice", 28), ("Bob", 35), ("Charlie", 42), ("David", 25)]
nomes_acima_30 = [nome for nome, idade in dados if (lambda x: x > 30)(idade)]


In [ ]:
from pyspark import SparkContext

sc = SparkContext("local", "Exemplo")

dados = [("Nova York", 25), ("Los Angeles", 30), ("Nova York", 28), ("Los Angeles", 32)]
rdd = sc.parallelize(dados)

resultado = (
    rdd
    .map(lambda x: (x[0], (x[1], 1))) 
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1])) 
    .map(lambda x: (x[0], x[1][0] / x[1][1]))  
    .sortBy(lambda x: x[1])  
    .collect()
)

print(resultado)


In [ ]:

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Exemplo").getOrCreate()
df = spark.createDataFrame([(1,), (2,), (3,), (4,), (5,)], ["valor"])
resultado = df.filter(df.valor % 2 == 0).select((df.valor * 2).alias("novo_valor")).agg(sum("novo_valor")).collect()[0][0]


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Exemplo").getOrCreate()

dados = [("Nova York", 25), ("Los Angeles", 30), ("Nova York", 28), ("Los Angeles", 32)]
df = spark.createDataFrame(dados, ["cidade", "temperatura"])

import pyspark.sql.functions as F

resultado = (
    df
    .groupBy("cidade")
    .agg(F.avg("temperatura").alias("media"))
    .orderBy("media")
    .collect()
)

resultado.show()


In [ ]:
vendas = [
    {"produto": "A", "categoria": "Eletrônicos", "valor": 1000},
    {"produto": "B", "categoria": "Roupas", "valor": 500},
    {"produto": "C", "categoria": "Eletrônicos", "valor": 800},
    {"produto": "D", "categoria": "Roupas", "valor": 300},
    {"produto": "E", "categoria": "Eletrônicos", "valor": 1200},
]


resultado = sorted(
    [(categoria, sum(venda["valor"] for venda in vendas if venda["categoria"] == categoria))
     for categoria in set(venda["categoria"] for venda in vendas)],
    key=lambda x: x[1],
    reverse=True
)

print(resultado)


[('Eletrônicos', 3000), ('Roupas', 800)]


In [ ]:
from collections import defaultdict


pedidos = [
    {"cliente": "Cliente1", "data": "2023-01-15", "valor": 100},
    {"cliente": "Cliente2", "data": "2023-01-20", "valor": 150},
    {"cliente": "Cliente1", "data": "2023-02-10", "valor": 200},
    {"cliente": "Cliente2", "data": "2023-02-18", "valor": 120},
    {"cliente": "Cliente1", "data": "2023-03-05", "valor": 300},
    
]


def extrair_mes(data):
    return data.split("-")[1]


gastos_por_cliente_e_mes = defaultdict(lambda: defaultdict(int))

"""Media gastos mes"""

for pedido in pedidos:
    cliente = pedido["cliente"]
    mes = extrair_mes(pedido["data"])
    valor = pedido["valor"]
    gastos_por_cliente_e_mes[cliente][mes] += valor


"""Média gastos mes"""
media_de_gastos = {
    cliente: {
        mes: total_gastos / len(pedidos)
        for mes, total_gastos in meses.items()
    }
    for cliente, meses in gastos_por_cliente_e_mes.items()
}

print(media_de_gastos)


{'Cliente1': {'01': 20.0, '02': 40.0, '03': 60.0}, 'Cliente2': {'01': 30.0, '02': 24.0}}
